In [1]:
import mesa
from mesa.time import SimultaneousActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer
import pandas as pd
import numpy as np
import random
import math

In [2]:
cities = pd.read_csv("data/cities.csv")
demographic = pd.read_csv("data/demographic.csv")
turnout = pd.read_csv("data/turnout.csv")

In [3]:
total_population_by_city = demographic.groupby("city")["population"].sum()

In [4]:

class Agent(mesa.Agent):
    def __init__(self, id, model, sex, age_group, income, education):
        super().__init__(id, model)
        self.id = id
        self.sex = sex
        self.age_group = age_group
        self.income = income
        self.education = education
        self.state = "Registered" # Undecided, Registered, WillVote, Voted
        self.utility = 0
    
    def step(self):
        self.move()
        if self.state == "Registered":
             # TODO: Replace benefit and cost based on demographic data
            benefit = 1
            cost = random.uniform(0, 1)

            # gets surrounding 8 cells and sums those that intend to vote
            neighbor_voter_count = sum([1 for pos in self.model.grid.get_neighborhood(self.pos, include_center=False, moore=True) for n in self.model.grid.get_cell_list_contents([pos]) if n.state == "WillVote"])
            social_influence = neighbor_voter_count / 8
            print(self.id, "neighbor voter count:", neighbor_voter_count)
            self.utility = benefit - cost + social_influence
            if self.utility > 0.5: # the benefits outweigh the costs
                self.state = "WillVote"
    
    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=False, # left, up, down, right only. no diagonals
            include_center=True # can stay in place
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

In [ ]:
scale = 200
class Model(mesa.Model):
    # Add parameters that you can adjust to see how it influences voter turnout
    def __init__(self, width=30, height=30):
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        self.running = True
        self.datacollector = DataCollector(
            model_reporters={
                "Total number of agents that intend to vote": lambda m: sum([1 for a in m.schedule.agents if a.state == "WillVote"])
            }
        )
        self.num_agents = 0
        total_count = sum([turnout[turnout["city"] == row["city"]]["registered"].values[0] * int(row["population"]) / total_population_by_city[row["city"]] for _, row in demographic.iterrows()])
        
        for _, row in demographic.iterrows():
            city = row["city"]
            age_group = row["age_group"]
            sex = row["sex"]
            # TODO: Add the proportion of income group and education group based on the data
            income = 0
            education = 0

            proportion = int(row["population"]) / total_population_by_city[city]
            count = turnout[turnout["city"] == city]["registered"].values[0] * proportion
            
            for __ in range(math.ceil(scale * count / total_count)):
                agent = Agent(self.num_agents, self, sex, age_group, income, education)
                self.schedule.add(agent)
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                self.grid.place_agent(agent, (x,y))
                self.num_agents += 1
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


In [6]:
# model = 
# simulated_turnout = {}
# for agent in model.

In [7]:
def agent_portrayal(a):
    portrayal = {
        "Shape": "circle",
        "Filled": "true",
        "r": 1
    }
    if a.state == "Undecided":
        portrayal["Color"] = "gray"
        portrayal["Layer"] = 0
    elif a.state == "Registered":
        portrayal["Color"] = "gray"
        portrayal["Layer"] = 1
    elif a.state == "WillVote":
        portrayal["Color"] = "blue"
        portrayal["Layer"] = 2
    else:
        portrayal["Color"] = "green"
        portrayal["Layer"] = 3
    return portrayal

grid = CanvasGrid(agent_portrayal, 30, 30, 800, 800)
# TODO: Label axes of the chart
chart = ChartModule([{
    "Label": "Total number of agents that intend to vote",
    "Color": "black"
}], data_collector_name="datacollector")

In [8]:

model = Model()
for i in range(10):
    print(f"Step {i+1}")
    model.step()

7322361.000000002
53643.16293684366 1.465187606479485
10906.577717858501 0.2978978424543258
12468.593300428234 0.3405621028634953
23794.47666028769 0.649912689644438
71162.56813571941 1.9437055380284964
4775.33237829153 0.13043149274643873
36667.6459553053 1.0015252172162856
12642.49470464257 0.3453119753216911
8005.734934940596 0.21866539863141393
21472.053513534684 0.5864789652827737
13737.718413374392 0.3752264717179169
19592.730340397957 0.5351478939756713
14329.904590160919 0.39140120488899455
16758.394497327503 0.45773199374703044
10705.247231400457 0.29239878316298407
23833.62894909566 0.6509820793893022
1816.5275269443634 0.04961589648323438
52897.01109713281 1.4448075176062145
10222.902991840434 0.2792242281373571
12540.633045285807 0.3425297672509128
23017.72799657353 0.6286968915237454
67937.45624564197 1.855616139265517
4512.398394896594 0.12324982051271695
36110.06408342883 0.9862956520015558
12910.430395614445 0.3526302621685667
8245.58434292224 0.22521654813037045
21062.

c:\Users\beaux\anaconda3\envs\consunji_beaux\lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)


18709.9694785217 0.5110365216498257
7014.488557823201 0.19159089692035666
3877.159592474843 0.1058991653778021
12024.399603903896 0.32842957630479824
8434.18616411537 0.23036794181863932
13540.845848898376 0.36984917430043046
8957.624506619804 0.24466492451327657
10329.294154135121 0.2821301532152026
7422.622757888239 0.20273850901063842
11376.857177334046 0.3107428649675711
1120.5030585493737 0.030604966309346764
29510.42798290616 0.8060358669261499
5815.603699754321 0.15884504191351176
6962.698514885135 0.19017632468230217
10960.470330283295 0.29936984342299683
36322.82266199737 0.9921068535680599
3126.5714263100826 0.08539790448217678
16919.638570183633 0.4621361489875637
6191.688575167796 0.16911727174248292
3474.772641520867 0.09490853131990805
10203.987403808229 0.2787075754338861
7360.70937319085 0.20104743191959118
12532.562434880943 0.3423093298699952
7767.290158157679 0.2121526146596071
9212.803487576668 0.25163477975414394
6570.27016992847 0.17945769595157812
9730.8869273571

In [9]:
server = ModularServer(
    Model,
    [grid, chart],
    "NCR Voter Turnout Model",
    {}
)
server.port = 4000

7322361.000000002
53643.16293684366 1.465187606479485
10906.577717858501 0.2978978424543258
12468.593300428234 0.3405621028634953
23794.47666028769 0.649912689644438
71162.56813571941 1.9437055380284964
4775.33237829153 0.13043149274643873
36667.6459553053 1.0015252172162856
12642.49470464257 0.3453119753216911
8005.734934940596 0.21866539863141393
21472.053513534684 0.5864789652827737
13737.718413374392 0.3752264717179169
19592.730340397957 0.5351478939756713
14329.904590160919 0.39140120488899455
16758.394497327503 0.45773199374703044
10705.247231400457 0.29239878316298407
23833.62894909566 0.6509820793893022
1816.5275269443634 0.04961589648323438
52897.01109713281 1.4448075176062145
10222.902991840434 0.2792242281373571
12540.633045285807 0.3425297672509128
23017.72799657353 0.6286968915237454
67937.45624564197 1.855616139265517
4512.398394896594 0.12324982051271695
36110.06408342883 0.9862956520015558
12910.430395614445 0.3526302621685667
8245.58434292224 0.22521654813037045
21062.

In [ ]:
server.launch()

Interface starting at http://127.0.0.1:4000


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
7322361.000000002
53643.16293684366 1.465187606479485
10906.577717858501 0.2978978424543258
12468.593300428234 0.3405621028634953
23794.47666028769 0.649912689644438
71162.56813571941 1.9437055380284964
4775.33237829153 0.13043149274643873
36667.6459553053 1.0015252172162856
12642.49470464257 0.3453119753216911
8005.734934940596 0.21866539863141393
21472.053513534684 0.5864789652827737
13737.718413374392 0.3752264717179169
19592.730340397957 0.5351478939756713
14329.904590160919 0.39140120488899455
16758.394497327503 0.45773199374703044
10705.247231400457 0.29239878316298407
23833.62894909566 0.6509820793893022
1816.5275269443634 0.04961589648323438
52897.01109713281 1.4448075176062145
10222.902991840434 0.2792242281373571
12540.633045285807 0.3425297672509128
23017.72799657353 0.6286968915237454
67937.45624564197 1.855616139265517
4512.398394896594 0.12324982051271695
36110.06408342883 0.9862956520015558
12910.430395614445 0.3526302621685667
8245.584342